In [1]:
!git clone https://github.com/openai/CLIP.git

fatal: destination path 'CLIP' already exists and is not an empty directory.


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ------------------------------------ 106.8/106.8 kB 772.8 kB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:
      Successfully uninstalled traitlets-5.0.5


In [1]:
import torch
print(torch.__version__)  #注意是双下划线

1.8.0+cpu


In [6]:
from pathlib import Path
photos_path = Path(r"E:\研究生\课程\研一下\互联网信息\project\image_dataset\image_dataset")
photos_files = list(photos_path.glob("*.png"))

In [7]:
print(f"Photos found: {len(photos_files)}")

Photos found: 3077


In [4]:
!chdir

F:\anacondafile\wsm_project


In [6]:
import sys
sys.executable

'D:\\codingapps\\anaconda\\python.exe'

In [9]:
!pip install .\CLIP

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing f:\anacondafile\wsm_project\clip
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369454 sha256=0873b3533c145830b203463c39e3863bccbc31255e2d0a86ce5d8daef13588f0
  Stored in directory: C:\Users\13309\AppData\Local\Temp\pip-ephem-wheel-cache-5jpiggz5\wheels\fa\4b\2b\095eb68fbfc0d36b09643be7e6d8806b1850ba64c258f8e0a7
Successfully built clip


In [2]:
import clip
from PIL import Image

# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B-32.pt", device=device)

In [2]:
def compute_clip_features(photos_batch):
    # Load all the photos from the files
    photos = [Image.open(photo_file) for photo_file in photos_batch]
    
    # Preprocess all photos
    photos_preprocessed = torch.stack([preprocess(photo) for photo in photos]).to(device)

    with torch.no_grad():
        # Encode the photos batch to compute the feature vectors and normalize them
        photos_features = model.encode_image(photos_preprocessed)
        photos_features /= photos_features.norm(dim=-1, keepdim=True)

    # Transfer the feature vectors back to the CPU and convert to numpy
    return photos_features.cpu().numpy()

In [7]:
import math
import numpy as np
import pandas as pd

# Define the batch size so that it fits on your GPU. You can also do the processing on the CPU, but it will be slower.
batch_size = 16

# Path where the feature vectors will be stored
features_path = Path(r"E:\研究生\课程\研一下\互联网信息\project\image_dataset\features")

# Compute how many batches are needed
batches = math.ceil(len(photos_files) / batch_size)

# Process each batch
for i in range(batches):
    print(f"Processing batch {i+1}/{batches}")

    batch_ids_path = features_path / f"{i:010d}.csv"
    batch_features_path = features_path / f"{i:010d}.npy"
    
    # Only do the processing if the batch wasn't processed yet
    if not batch_features_path.exists():
        try:
            # Select the photos for the current batch
            batch_files = photos_files[i*batch_size : (i+1)*batch_size]

            # Compute the features and save to a numpy file
            batch_features = compute_clip_features(batch_files)
            np.save(batch_features_path, batch_features)

            # Save the photo IDs to a CSV file
            photo_ids = [photo_file.name.split(".")[0] for photo_file in batch_files]
            photo_ids_data = pd.DataFrame(photo_ids, columns=['photo_id'])
            photo_ids_data.to_csv(batch_ids_path, index=False)
        except:
            # Catch problems with the processing to make the process more robust
            print(f'Problem with batch {i}')

Processing batch 1/193
Processing batch 2/193
Processing batch 3/193
Processing batch 4/193
Processing batch 5/193
Processing batch 6/193
Processing batch 7/193
Processing batch 8/193
Processing batch 9/193
Processing batch 10/193
Processing batch 11/193
Processing batch 12/193
Processing batch 13/193
Processing batch 14/193
Processing batch 15/193
Processing batch 16/193
Processing batch 17/193
Processing batch 18/193
Processing batch 19/193
Processing batch 20/193
Processing batch 21/193
Processing batch 22/193
Processing batch 23/193
Processing batch 24/193
Processing batch 25/193
Processing batch 26/193
Processing batch 27/193
Processing batch 28/193
Processing batch 29/193
Processing batch 30/193
Processing batch 31/193
Processing batch 32/193
Processing batch 33/193
Processing batch 34/193
Processing batch 35/193
Processing batch 36/193
Processing batch 37/193
Processing batch 38/193
Processing batch 39/193
Processing batch 40/193
Processing batch 41/193
Processing batch 42/193
P

In [8]:
import numpy as np
import pandas as pd

# Load all numpy files
features_list = [np.load(features_file) for features_file in sorted(features_path.glob("*.npy"))]

# Concatenate the features and store in a merged file
features = np.concatenate(features_list)
np.save(features_path / "features.npy", features)

# Load all the photo IDs
photo_ids = pd.concat([pd.read_csv(ids_file) for ids_file in sorted(features_path.glob("*.csv"))])
photo_ids.to_csv(features_path / "photo_ids.csv", index=False)